In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt

from floriswakemap import AreaSelector
wm05 = __import__('05a_run_large_WakeMap')

#### Load the data generated by example 05a

Construct the `WakeMap` and assign the saved results to it

In [ ]:
# Take a couple of minutes because the gridded existing layouts are regenerated.
filename = "example_05_raw_expected_powers.npz"
wake_map = wm05.construct_example_05_WakeMap()
wake_map.load_raw_expected_powers(filename)
print(
    "Shape of loaded expected_powers_candidates:",
    wake_map.expected_powers_candidates_raw.shape
)
print(
    "Shape of loaded expected_powers_existing:",
    wake_map.expected_powers_existing_raw.shape
)

save_figs = False
figdim = 10 # Dimension for figures

if save_figs and not os.path.exists("figs"):
    os.makedirs("figs")

Visualize the wind rose and domain layout

In [ ]:
# Plot the wind rose, as well the positions of the existing turbines, boundaries, etc.
ax = wake_map.fmodel_existing.wind_data.plot()
fig = ax.get_figure()
fig.set_size_inches(figdim, figdim)
if save_figs:
    fig.savefig("figs/ex05-1_wind_rose.png", dpi=300, bbox_inches="tight", format="png")

fig, ax = plt.subplots(figsize=(figdim, figdim))
ax.set_aspect('equal')
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize": 5})
wake_map.plot_candidate_boundary(ax=ax, color="green")
wake_map.plot_exclusion_zones(ax=ax, color="orange")

ax.set_xlabel("X location [m]")
ax.set_ylabel("Y location [m]")

# Plot also all of the candidate locations, as well as showing the candidate farm in one
# specific location.
wake_map.plot_candidate_locations(
    ax=ax,
    plotting_dict={"markersize": 5, "color": "green", "alpha": 0.1}
)
wake_map.plot_candidate_layout(100, ax=ax, plotting_dict={"markersize": 10, "color": "red"})

if save_figs:
    fig.savefig("figs/ex05-2_layout.png", dpi=300, bbox_inches="tight", format="png")

#### Generate WakeMaps

Process the data and return values

In [ ]:
# Expected powers
existing_expected_powers = wake_map.process_existing_expected_powers()
candidate_expected_powers = wake_map.process_candidate_expected_powers()

# AEP losses
existing_aep_loss = wake_map.process_existing_aep_loss()
candidate_aep_loss = wake_map.process_candidate_aep_loss()

# Create a pandas dataframe with the results. These won't be used again, but
# may be useful for users to see.
df_processed = pd.DataFrame(
    data={
        "candidate_cluster_x": wake_map.all_candidates_x,
        "candidate_cluster_y": wake_map.all_candidates_y,
        "EP_existing": existing_expected_powers,
        "AEPL_existing": existing_aep_loss,
        "EP_candidates": candidate_expected_powers,
        "AEPL_candidates": candidate_aep_loss,
    }
)
print(df_processed.head())

Generate maps of AEP losses of candidate clusters and existing locations

In [ ]:
# Plot candidate map
fig, ax = plt.subplots(1,1, figsize=(figdim,figdim))
ax.set_aspect("equal")
wake_map.plot_candidate_value(ax=ax, value="aep_loss")
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize": 5})
wake_map.plot_exclusion_zones(ax=ax, color="orange")
if save_figs:
    fig.savefig("figs/eg_05-3_candidates_aeploss.png", dpi=300, bbox_inches='tight', format="png")

# Plot existing map
fig, ax = plt.subplots(1,1, figsize=(figdim,figdim))
ax.set_aspect("equal")
wake_map.plot_existing_value(ax=ax, value="aep_loss")
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize": 5})
wake_map.plot_exclusion_zones(ax=ax, color="orange")
if save_figs:
    fig.savefig("figs/eg_05-3_existing_aeploss.png", dpi=300, bbox_inches='tight', format="png")

Maps of AEP loss at existing farm subsets

In [ ]:
n_turbs_1 = 131
n_turbs_2 = 274
n_turbs_3 = 188
subset_1 = range(n_turbs_1)
subset_2 = range(n_turbs_1, n_turbs_1+n_turbs_2)
subset_3 = range(n_turbs_1+n_turbs_2, n_turbs_1+n_turbs_2+n_turbs_3)

for i, subset in enumerate([subset_1, subset_2, subset_3]):
    fig, ax = plt.subplots(1,1, figsize=(figdim,figdim))
    ax.set_aspect("equal")
    wake_map.plot_existing_value(ax=ax, value="aep_loss", subset=subset)
    wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize": 5, "color": "lightgray"})
    wake_map.plot_existing_farm(
        ax=ax,
        subset=subset,
        plotting_dict={"color": "black", "markersize": 5}
    )
    wake_map.plot_exclusion_zones(ax=ax, color="orange")
    if save_figs:
        fig.savefig(
            "figs/eg_05-4_existing_aeploss_subset{0}.png".format(i),
            dpi=300, bbox_inches='tight', format="png"
        )


### Area selection

Generate the `AreaSelector` object

In [ ]:
area_selector = AreaSelector(wake_map, verbose=True)

Add constraints based on AEP loss thresholds and select candidates

In [ ]:
area_selector.reset_constraints() # Only needed if cell is run multiple times

# Constraint for AEP loss at candidate group
area_selector.add_constraint({
    "turbines": "candidates",
    "value": "aep_loss",
    "threshold": 100,
    "name": "candidates_aep_loss"
})

area_selector.select_candidates()
fig, ax = plt.subplots(1,1, figsize=(figdim,figdim))
wake_map.plot_candidate_value(ax=ax, value="aep_loss")
area_selector.plot_constraints(ax=ax)
ax.set_aspect("equal")
#wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":ms*4, "color":"white"})
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":5})
wake_map.plot_exclusion_zones(ax=ax, color="orange")
if save_figs:
    fig.savefig(
        "figs/example_05-5_candidate_constraint.png",
        dpi=300, bbox_inches="tight", format="png"
    )


# Constraint for AEP loss at existing farms
area_selector.add_constraint({
    "turbines": "existing",
    "value": "aep_loss",
    "threshold": 60,
    "name": "existing_aep_loss"
})

area_selector.select_candidates()
fig, ax = plt.subplots(1,1, figsize=(figdim,figdim))
wake_map.plot_existing_value(ax=ax, value="aep_loss")
area_selector.plot_constraints(ax=ax, to_plot=["existing_aep_loss"])
ax.set_aspect("equal")
#wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":ms*4, "color":"white"})
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":5})
wake_map.plot_exclusion_zones(ax=ax, color="orange")
if save_figs:
    fig.savefig(
        "figs/example_05-6_existing_constraint.png",
        dpi=300, bbox_inches="tight", format="png"
    )

Add a combined objective and make selection

In [ ]:
# Remake the figure
fig, ax = plt.subplots(1,1, figsize=(figdim,figdim))
wake_map.plot_existing_value(ax=ax, value="aep_loss")
area_selector.plot_constraints(ax=ax)
ax.set_aspect("equal")
wake_map.plot_exclusion_zones(ax=ax, color="orange")

# Add an evenly-weighted objective and select the best 100 locations
area_selector.add_objective({
    "value": "aep_loss",
    "candidates_weight": 0.5,
    "existing_weight": 0.5,
    "n_target": 100
})
area_selector.select_candidates()

# Plot the selected 100 locations overtop of the existing farm AEP loss map
ax = area_selector.plot_selection(ax=ax, plotting_dict={"markersize":5, "color":"blue"})
#wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":ms*4, "color":"white"})
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":5})
if save_figs:
    fig.savefig(
        "figs/example_05-7a_objective_existing.png",
        dpi=300, bbox_inches="tight", format="png"
    )


# Plot the selected 100 locations overtop of the candidate farm AEP loss map
fig, ax = plt.subplots(1,1, figsize=(figdim, figdim))
wake_map.plot_candidate_value(ax=ax, value="aep_loss")
area_selector.plot_constraints(ax=ax)
ax.set_aspect("equal")
wake_map.plot_exclusion_zones(ax=ax, color="orange")
ax = area_selector.plot_selection(ax=ax, plotting_dict={"markersize":5, "color":"blue"})
#wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":ms*4, "color":"white"})
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":5})
if save_figs:
    fig.savefig(
        "figs/example_05-7b_objective_candidate.png",
        dpi=300, bbox_inches="tight", format="png"
    )


# Plot the selected 100 locations overtop of the combined objective function value
fig, ax = plt.subplots(1,1, figsize=(figdim, figdim))
area_selector.plot_objective(ax=ax, cmap="gist_yarg")
area_selector.plot_constraints(ax=ax)
ax.set_aspect("equal")
#wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":ms*4, "color":"white"})
wake_map.plot_existing_farm(ax=ax, plotting_dict={"markersize":5})
wake_map.plot_exclusion_zones(ax=ax, color="orange")
ax = area_selector.plot_selection(ax=ax, plotting_dict={"markersize":5, "color":"blue"})
if save_figs:
    fig.savefig(
        "figs/example_05-7c_objective_combined.png",
        dpi=300, bbox_inches="tight", format="png"
    )